In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
df = pd.read_csv('aggregated_hourly.csv')
#for every distinct device_id create a new column target with the value of WIFI shiftied by 1
df['target'] = 0
for i in df['device_id'].unique():
    df.loc[df['device_id'] == i, 'target'] = df.loc[df['device_id'] == i, 'CO2'].shift(-1)
df.sort_values(by=['date_time'], inplace=True)
#drop
df = df.dropna()   
#onehot encode device_id into int
df = pd.get_dummies(df, columns=['device_id'], prefix = 'device_id')
#delete date_time column and device_id column
df = df.drop(columns=['date_time'])
df

,tmp,hum,snr,CO2,VOC,vis,IR,WIFI,BLE,rssi,...,device_id_hka-aqm-am201a,device_id_hka-aqm-am201b,device_id_hka-aqm-am204,device_id_hka-aqm-am205,device_id_hka-aqm-am209,device_id_hka-aqm-am210,device_id_hka-aqm-am211,device_id_hka-aqm-am301,device_id_hka-aqm-am307,device_id_hka-aqm-am308
46711,25.080000,44.9700,-16.800000,754,558,379,64,4,0,-131,...,False,False,False,False,False,False,False,False,False,False
54194,23.900000,52.1100,-15.200000,686,593,255,35,5,0,-135,...,False,False,False,False,False,False,False,False,False,False
54195,24.137500,51.8300,-11.625000,800,633,256,36,3,0,-125,...,False,False,False,False,False,False,False,False,False,False
54196,24.432500,52.0000,-15.650000,902,825,289,55,5,2,-125,...,False,False,False,False,False,False,False,False,False,False
24671,24.908000,52.3160,-2.550000,1128,450,213,91,3,1,-115,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24669,24.595000,45.5850,-14.900000,469,790,5,1,2,1,-125,...,False,False,False,False,False,False,False,False,False,False
84148,27.086667,38.9300,9.066667,441,836,32,4,5,0,-102,...,False,False,False,False,False,False,False,False,False,False
61166,24.035000,48.9550,6.000000,10026,9997,12,3,2,2,-96,...,False,False,False,False,False,False,False,False,False,False
7473,24.492500,45.9275,-9.700000,445,865,8,2,1,4,-120,...,False,False,False,False,False,False,False,False,False,False


In [3]:
X = df.drop(columns=['target']).astype('float32')
y = df['target'].astype('float32')
X = torch.tensor(X.values).float()
y = torch.tensor(y.values).float()
print(X.shape)  
print(y.shape)

torch.Size([143802, 58])
torch.Size([143802])


In [4]:
X_new = []
window_size = 500
#for i in range(0, len(X), window_size):
for i in range(0, len(X)):
    if i < window_size:
        number_of_padding = window_size - i
        padding = torch.zeros(number_of_padding, X.shape[1])
        X_new.append(torch.cat((padding, X[:i])))
    else:
        X_new.append(X[i-window_size:i])
X_new = torch.stack(X_new)
X_new = X_new[1:]
X_new.shape

torch.Size([143801, 500, 58])

In [5]:
from torch.utils.data import DataLoader, TensorDataset
batch_size=6
# Create a TensorDataset
y = y[1:]
data = TensorDataset(X_new, y)

from torch.utils.data import DataLoader, TensorDataset
# Create a DataLoader
data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [ ]:
class RNN(nn.Module):

    def __init__(self, input_size= X_new.shape[2], hidden_size=26, output_size=1, num_layers=1):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out
    
    def train(self, x, y, num_epochs, learning_rate):
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for epoch in range(num_epochs):
            outputs = self(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [9]:
RNN_model = RNN(input_size=X_new.shape[2], hidden_size=26, output_size=1, num_layers=1)
RNN_model.train(X_new, y, num_epochs=100, learning_rate=0.01)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([143801])) that is different to the input size (torch.Size([143801, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


: 